# Modelos de IA para NLP

In [ ]:
import warnings
warnings.filterwarnings('ignore')

___
### PyTorch/Embedding and EmbeddingBag
[site](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html)

Embedding é uma classe que representa uma camada de embedding. Ela aceita índices de token e produz vetores de embedding. EmbeddingBag é uma classe que agrega embeddings usando operações de média ou soma. Embedding e EmbeddingBag são parte do módulo torch.nn. 

O exemplo de código mostra como você pode usar Embedding e EmbeddingBag no PyTorch.

In [11]:
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

import torch
import torch.nn as nn

In [1]:
# Defining a data set
dataset = [
"I like cats",
"I hate dogs",
"I'm impartial to hippos"
]

In [6]:
#Initializing the tokenizer, iterator from the data set, and vocabulary
tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
def yield_tokens(data_iter):
    for data_sample in data_iter:
        yield tokenizer(data_sample)
data_iter = iter(dataset)
vocab = build_vocab_from_iterator(yield_tokens(data_iter))

In [9]:
#Tokenizing and generating indices
input_ids=lambda x:[torch.tensor(vocab(tokenizer(data_sample))) for data_sample in dataset]
index=input_ids(dataset)
print(index)

[tensor([0, 7, 2]), tensor([0, 4, 3]), tensor([0, 1, 6, 8, 5])]


In [12]:
#Initiating the embedding layer, specifying the dimension size for the embeddings, 
#determining the count of unique tokens present in the vocabulary, and creating the embedding layer
embedding_dim = 3
n_embedding = len(vocab)
n_embedding:9
embeds = nn.Embedding(n_embedding, embedding_dim)

In [13]:
#Applying the embedding object
i_like_cats=embeds(index[0])
i_like_cats
impartial_to_hippos=embeds(index[-1])
impartial_to_hippos

tensor([[-0.4136, -0.1322, -0.8446],
        [ 1.0880, -1.7253, -0.0407],
        [-0.2187, -1.1263, -1.0967],
        [ 1.1725,  0.0905,  1.0090],
        [ 0.5685, -1.2016,  1.2934]], grad_fn=<EmbeddingBackward0>)

In [14]:
#Initializing the embedding bag layer
embedding_dim = 3
n_embedding = len(vocab)
n_embedding:9
embedding_bag = nn.EmbeddingBag(n_embedding, embedding_dim)

In [16]:
# Output the embedding bag
dataset = ["I like cats","I hate dogs","I'm impartial to hippos"]
i_like_cats=embedding_bag(index[0],offsets=torch.tensor([0]))
i_like_cats

tensor([[ 1.0653, -0.2848, -1.0250]], grad_fn=<EmbeddingBagBackward0>)

___
### Batch function

Define o número de amostras que serão propagadas pela rede.

In [18]:
from torch.utils.data import DataLoader

In [17]:
def collate_batch(batch):
    target_list, context_list, offsets = [], [], [0]
    for _context, _target in batch:
        target_list.append(vocab[_target]) 
        processed_context = torch.tensor(text_pipeline(_context), dtype=torch.int64)
        context_list.append(processed_context)
        offsets.append(processed_context.size(0))
        target_list = torch.tensor(target_list, dtype=torch.int64)
        offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
        context_list = torch.cat(context_list)
    return target_list.to(device), context_list.to(device), offsets.to(device)

In [24]:
toy_data = """I wish I was little bit taller
I wish I was a baller
She wore a small black dress to the party
The dog chased a big red ball in the park
He had a huge smile on his face when he won the race
The tiny kitten played with a fluffy toy mouse
The team celebrated their victory with a grand parade
She bought a small, delicate necklace for her sister
The mountain peak stood majestic and tall against the clear blue sky
The toddler took small, careful steps as she learned to walk
The house had a spacious backyard with a big swimming pool
He felt a sense of accomplishment after completing the challenging puzzle
The chef prepared a delicious, flavorful dish using fresh ingredients
The children played happily in the small, cozy room
The book had an enormous impact on readers around the world
The wind blew gently, rustling the leaves of the tall trees
She painted a beautiful, intricate design on the small canvas
The concert hall was filled with thousands of excited fans
The garden was adorned with colorful flowers of all sizes
I hope to achieve great success in my chosen career path
The skyscraper towered above the city, casting a long shadow
He gazed in awe at the breathtaking view from the mountaintop
The artist created a stunning masterpiece with bold brushstrokes
The baby took her first steps, a small milestone that brought joy to her parents
The team put in a tremendous amount of effort to win the championship
The sun set behind the horizon, painting the sky in vibrant colors
The professor gave a fascinating lecture on the history of ancient civilizations
The house was filled with laughter and the sound of children playing
She received a warm, enthusiastic welcome from the audience
The marathon runner had incredible endurance and determination
"""

In [25]:
tokenizer = get_tokenizer('basic_english')

def tokenize_data(sentences):
    for sentence in sentences:
        yield tokenizer(sentence)

tokenized_toy_data = tokenizer(toy_data)

# slide over the sequence and create training data:
CONTEXT_SIZE = 2

cobow_data = []
for i in range(1, len(tokenized_toy_data) - CONTEXT_SIZE):
    context = (
        [tokenized_toy_data[i - j -1] for j in range(CONTEXT_SIZE)]
        + [tokenized_toy_data[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = tokenized_toy_data[i]
    cobow_data.append((context, target))

In [26]:
BATCH_SIZE = 64 # batch size for training
dataloader_cbow = DataLoader(cobow_data, 
                             batch_size=BATCH_SIZE, 
                             shuffle=True, 
                             collate_fn=collate_batch)

___
### Passe para frente (Forward pass)

Refere-se ao cálculo e armazenamento de variáveis ​​intermediárias (incluindo saídas) para uma rede neural, da camada de entrada para a camada de saída.

In [27]:
class CBOW(nn.Module):
    # Initialize the CBOW model
    def __init__(self, vocab_size, embed_dim, num_class):
        super(CBOW, self).__init__()

        # Define the embedding layer using nn.EmbeddingBag
        # It outputs the average of context words embeddings
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse = False)

        # Define the first linear layer with input size embed_dim and output size embed_dim/2
        self.linear1 = nn.Linear(embed_dim, embed_dim//2)

        # Define the fully connected layer with input size embed_dim/2 and output size vocab_size
        self.fc = nn.Linear(embed_dim//2, vocab_size)

        self.init_weights()

    # Initialize the weights of the model's parameters
    def init_weights(self):
        # Initialize the weights of the embedding layer
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)

        # Initialize the weights of the fully connected layer
        self.fc.weight.data.uniform_(-initrange, initrange)

        # Initialize the biases of the fully connected layer to zeros
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        # Pass the input text and offsets through the embeddings layer
        out = self.embedding(text, offsets)

        # Apply the ReLU activation function to the output of the first linear layer
        out = torch.relu(self.linear1(out))

        # Pass the output of the ReLU activation through the fully connected layer
        return self.fc(out)

___
### GloVe pré-treinado de Stanford
[site](https://nlp.stanford.edu/projects/glove/)

Aproveita dados em larga escala para embeddings de palavras. Pode ser integrado ao PyTorch para tarefas de PNL aprimoradas, como classificação.

In [28]:
from torchtext.vocab import GloVe,vocab

In [29]:
# Creating an instance of the 6B version of Glove() model
glove_vectors_6B = GloVe(name ='6B') # you can specify the model with the following format: GloVe(name='840B', dim=300)

.vector_cache\glove.6B.zip: 862MB [03:35, 4.00MB/s]                                                                    
100%|███████████████████████████████████████████████████████████████████████▉| 399999/400000 [00:49<00:00, 8078.69it/s]


In [30]:
# Build vocab from glove_vectors
vocab = vocab(glove_vectors_6B.stoi, 0,specials=('<unk>', '<pad>'))
vocab.set_default_index(vocab["<unk>"])

___
### vocab
[site](https://pytorch.org/text/main/vocab.html)

O objeto vocab faz parte da biblioteca torchtext do PyTorch. Ele mapeia tokens para índices. O exemplo de código mostra como você pode aplicar o objeto vocab a tokens diretamente.

In [33]:
class SentenceIterator:
    def __init__(self, text):
        # Divide o texto em sentenças
        self.sentences = text.split(".")
        self.index = 0  # Índice para rastrear a próxima sentença

    def __iter__(self):
        # Retorna o próprio iterador
        return self

    def __next__(self):
        # Retorna a próxima sentença ou levanta StopIteration
        if self.index < len(self.sentences):
            sentence = self.sentences[self.index].strip()
            self.index += 1
            return sentence
        else:
            raise StopIteration

In [34]:
# Exemple
text = "Python é uma linguagem poderosa. Ela é amplamente usada em IA. Fácil de aprender."
my_iterator = SentenceIterator(text)

In [35]:
# Takes an iterator as input and extracts the next tokenized sentence. Creates a list of token indices using the vocab dictionary for each token.
def get_tokenized_sentence_and_indices(iterator):
    tokenized_sentence = next(iterator)
    token_indices = [vocab[token] for token in tokenized_sentence]
    return tokenized_sentence, token_indices

In [36]:
# Returns the tokenized sentences and the corresponding token indices. Repeats the process.
tokenized_sentence, token_indices = get_tokenized_sentence_and_indices(my_iterator)
next(my_iterator)

'Ela é amplamente usada em IA'

In [37]:
# Prints the tokenized sentence and its corresponding token indices.
print("Tokenized Sentence:", tokenized_sentence)
print("Token Indices:", token_indices)

Tokenized Sentence: Python é uma linguagem poderosa
Token Indices: [0, 3526, 2161, 5920, 4870, 3816, 0, 67614, 0, 6481, 1995, 9, 0, 5027, 43, 3816, 3412, 6481, 9, 3412, 1112, 1995, 0, 3422, 4870, 1970, 1112, 1913, 4870, 1536, 9]


___
### Special tokens in PyTorch: `<eos>` and `<bos>`


Tokens introduzidos em sequências de entrada para transmitir informações específicas ou atender a um propósito particular durante o treinamento. O exemplo de código mostra o uso de `<bos>` e `<eos>` durante a tokenização. O token `<bos>` denota o início da sequência de entrada, e o token `<eos>` denota o fim.

In [44]:
# Exemplo de texto
text = """
Python é uma linguagem de programação poderosa.
É amplamente usada em inteligência artificial.
Este é um exemplo simples para demonstrar a tokenização.
"""

# `lines` é uma lista de strings, onde cada string é uma linha do texto
lines = text.strip().split("\n")

In [45]:
# Appends <bos> at the beginning and <eos> at the end of the tokenized sentences 
# using a loop that iterates over the sentences in the input data
tokenizer_en = get_tokenizer('spacy', language='en_core_web_sm')
tokens = []
max_length = 0
for line in lines:
    tokenized_line = tokenizer_en(line)
    tokenized_line = ['<bos>'] + tokenized_line + ['<eos>']
    tokens.append(tokenized_line)
    max_length = max(max_length, len(tokenized_line))

In [46]:
tokens[:3]

[['<bos>',
  'Python',
  'é',
  'uma',
  'linguagem',
  'de',
  'programação',
  'poderosa',
  '.',
  '<eos>'],
 ['<bos>',
  'É',
  'amplamente',
  'usada',
  'em',
  'inteligência',
  'artificial',
  '.',
  '<eos>'],
 ['<bos>',
  'Este',
  'é',
  'um',
  'exemplo',
  'simples',
  'para',
  'demonstrar',
  'a',
  'tokenização',
  '.',
  '<eos>']]

___
### Special tokens in PyTorch: `<pad>` 


Tokens introduzidos em sequências de entrada para transmitir informações específicas ou atender a um propósito particular durante o treinamento. O exemplo de código mostra o uso do token `<pad>` para garantir que todas as sentenças tenham o mesmo comprimento.

In [47]:
# Pads the tokenized lines
for i in range(len(tokens)):
    tokens[i] = tokens[i] + ['<pad>'] * (max_length - len(tokens[i]))

In [48]:
tokens[:3]

[['<bos>',
  'Python',
  'é',
  'uma',
  'linguagem',
  'de',
  'programação',
  'poderosa',
  '.',
  '<eos>',
  '<pad>',
  '<pad>'],
 ['<bos>',
  'É',
  'amplamente',
  'usada',
  'em',
  'inteligência',
  'artificial',
  '.',
  '<eos>',
  '<pad>',
  '<pad>',
  '<pad>'],
 ['<bos>',
  'Este',
  'é',
  'um',
  'exemplo',
  'simples',
  'para',
  'demonstrar',
  'a',
  'tokenização',
  '.',
  '<eos>']]

___
### Perda de entropia cruzada (Cross entropy loss) 

Uma métrica usada em machine learning (ML) para avaliar o desempenho de um modelo de classificação. A perda é medida como o valor de probabilidade entre 0 (modelo perfeito) e 1. Normalmente, o objetivo é trazer o modelo o mais próximo possível de 0.

#### O que é nn.CrossEntropyLoss
A função CrossEntropyLoss combina duas operações:

- Log-Softmax: Converte os logits (saída da última camada do modelo) em probabilidades logarítmicas.
- Negative Log-Likelihood (NLL): Calcula a distância entre as distribuições de probabilidade predita pelo modelo e os rótulos verdadeiros.

Isso significa que você pode passar diretamente os logits para CrossEntropyLoss, sem aplicar softmax antes.

In [51]:
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss

In [52]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_size, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_size, sparse=True)
        self.fc = nn.Linear(embed_size, num_class)
        self.init_weights()

    def init_weights(self):
        # Inicializa os pesos das camadas
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        # Calcula a representação embutida
        embedded = self.embedding(text, offsets)
        # Passa pela camada linear para prever as classes
        return self.fc(embedded)

In [53]:
# Parâmetros do modelo
vocab_size = 5000  # Tamanho do vocabulário
embed_size = 64    # Dimensão dos embeddings
num_class = 3      # Número de classes de saída

# Exemplo de texto e offsets
text = torch.tensor([1, 2, 3, 4, 5, 6])  # Índices dos tokens
offsets = torch.tensor([0, 3])  # Offset indicando onde as sequências começam

In [57]:
# Exemplo de rótulos
label = torch.tensor([1, 0])  # Rótulos das sequências

In [58]:
model = TextClassificationModel(vocab_size,embed_size,num_class)
loss_fn = CrossEntropyLoss()
predicted_label = model(text, offsets)
criterion = nn.CrossEntropyLoss()
loss = criterion(predicted_label, label)

In [59]:
loss

tensor(1.0009, grad_fn=<NllLossBackward0>)

___
### Otimização (Optimization) 

Método para reduzir perdas em um modelo.

In [60]:
# Creates an iterator object
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
optimizer.zero_grad()

In [ ]:
predicted_label = model(text, offsets)
loss = criterion(predicted_label, label)
loss.backward()
torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
optimizer.step()

___
### sentence_bleu()
[site](https://www.nltk.org/_modules/nltk/translate/bleu_score.html)

NLTK (ou Natural Language Toolkit) fornece esta função para avaliar uma sentença de hipótese em relação a uma ou mais sentenças de referência. As sentenças de referência devem ser apresentadas como uma lista de sentenças onde cada referência é uma lista de tokens.

In [63]:
from nltk.translate.bleu_score import sentence_bleu

In [65]:
def calculate_bleu_score(generated_translation, reference_translations):
    # Convert the generated translations and reference translations into the expected format for sentence_bleu
    references = [reference.split() for reference in reference_translations]
    hypothesis = generated_translation.split()
    # Calculate the BLEU score
    bleu_score = sentence_bleu(references, hypothesis)
    
    return bleu_score

In [66]:
reference_translations = ["Asian man sweeping the walkway .",
                          "An asian man sweeping the walkway .",
                          "An Asian man sweeps the sidewalk .",
                          "An Asian man is sweeping the sidewalk .",
                          "An asian man is sweeping the walkway .",
                          "Asian man sweeping the sidewalk ."]

In [ ]:
bleu_score = calculate_bleu_score(generated_translation, 
                                  reference_translations)

___
### Modelo Encoder RNN
O modelo seq2seq codificador-decodificador trabalha em conjunto para transformar uma sequência de entrada em uma sequência de saída. O codificador é uma série de RNNs que processam a sequência de entrada individualmente, passando seus estados ocultos para sua próxima RNN.

In [68]:
class Encoder(nn.Module):
    def __init__(self, vocab_len, emb_dim, hid_dim, n_layers, dropout_prob):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(vocab_len, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout_prob)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, input_batch):
        embed = self.dropout(self.embedding(input_batch))
        embed = embed.to(device)
        outputs, (hidden, cell) = self.lstm(embed)
        return hidden, cell

___
### Modelo Decoder RNN

O modelo seq2seq codificador-decodificador trabalha em conjunto para transformar uma sequência de entrada em uma sequência de saída. O módulo decodificador é uma série de RNNs que geram autorregressivamente a tradução como um token por vez. Cada token gerado volta para o próximo RNN junto com o estado oculto para gerar o próximo token da sequência de saída até que o token final seja gerado.

In [69]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropout = nn.Dropout(dropout)
    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction_logit = self.fc_out(output.squeeze(0))
        prediction = self.softmax(prediction_logit)
        return prediction, hidden, cell

___
### Modelo Skip-gram

Prevê palavras de contexto circundantes a partir de uma palavra-alvo específica. Ele prevê uma palavra de contexto por vez a partir de uma palavra-alvo.

In [72]:
class SkipGram_Model(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(SkipGram_Model, self).__init__()
        # Define the embeddings layer
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)
        # Define the fully connected layer
        self.fc = nn.Linear(in_features=embed_dim, out_features=vocab_size)
        
    # Perform the forward pass
    def forward(self, text):
        # Pass the input text through the embeddings layer
        out = self.embeddings(text)
        # Pass the output of the embeddings layer through the fully connected layer
        # Apply the ReLU activation function
        out = torch.relu(out)
        out = self.fc(out)
        return out

___
### collate_fn

Processa a lista de amostras para formar um lote. O argumento `batch` é uma lista de todas as suas amostras.

In [76]:
def collate_fn(batch):
    target_list, context_list = [], []
    for _context, _target in batch:
        target_list.append(vocab[_target])
        context_list.append(vocab[_context])
        target_list = torch.tensor(target_list, dtype=torch.int64)
        context_list = torch.tensor(context_list, dtype=torch.int64)
    return target_list.to(device), context_list.to(device)

___
### Função de treinamento

Treina o modelo para um número especificado de épocas. Também inclui uma condição para verificar se a entrada é para skip-gram ou CBOW. A saída desta função inclui o modelo treinado e uma lista de perdas médias para cada época.

In [77]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=1000):
    # List to store running loss for each epoch
    epoch_losses = []

    for epoch in tqdm(range(num_epochs)):
        # Storing running loss values for the current epoch
        running_loss = 0.0

        # Loop through the data loader
        for idx, samples in enumerate(dataloader):
            optimizer.zero_grad()

            # Check for EmbeddingBag layer in the model (CBOW)
            if any(isinstance(module, nn.EmbeddingBag) for _, module in model.named_modules()):
                target, context, offsets = samples
                predicted = model(context, offsets)

            # Check for Embedding layer in the model (Skip-gram)
            elif any(isinstance(module, nn.Embedding) for _, module in model.named_modules()):
                target, context = samples
                predicted = model(context)

            # Compute the loss
            loss = criterion(predicted, target)

            # Backpropagation
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
            optimizer.step()

            # Update running loss
            running_loss += loss.item()

        # Append average loss for the epoch
        epoch_losses.append(running_loss / len(dataloader))

    return model, epoch_losses


___
### Modelo CBOW

Utiliza palavras de contexto para prever uma palavra-alvo e gerar sua incorporação.

In [78]:
class CBOW(nn.Module):
    # Initialize the CBOW model
    def __init__(self, vocab_size, embed_dim, num_class):
        super(CBOW, self).__init__()
        # Define the embedding layer using nn.EmbeddingBag
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        # Define the fully connected layer
        self.fc = nn.Linear(embed_dim, vocab_size)

    def forward(self, text, offsets):
        # Pass the input text and offsets through the embedding layer
        out = self.embedding(text, offsets)
        # Apply the ReLU activation function to the output of the embedding layer
        out = torch.relu(out)
        # Pass the output of the ReLU activation through the fully connected layer
        return self.fc(out)

In [79]:
# Example of using the CBOW model
vocab_size = len(vocab)  # Assuming 'vocab' is defined
emsize = 24  # Size of the embedding dimension
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the CBOW model
model_cbow = CBOW(vocab_size, emsize, vocab_size).to(device)

___
### Training loop

Enumera dados do DataLoader e, em cada passagem do loop, obtém um lote de dados de treinamento do DataLoader, zera os gradientes do otimizador e executa uma inferência (obtém previsões do modelo para um lote de entrada).

In [ ]:
for epoch in tqdm(range(1, EPOCHS + 1)):
    model.train()
    cum_loss=0
    for idx, (label, text, offsets) in enumerate(train_dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        cum_loss+=loss.item()
    cum_loss_list.append(cum_loss)
    accu_val = evaluate(valid_dataloader)
    acc_epoch.append(accu_val)
    if accu_val > acc_old:
        acc_old= accu_val
        torch.save(model.state_dict(), 'my_model.pth')

____
Esse material tem como referência o curso [Gen AI Foundational Models for NLP & Language Understanding](https://www.coursera.org/learn/gen-ai-foundational-models-for-nlp-and-language-understanding?specialization=generative-ai-engineering-with-llms)